## imports

In [1]:
import pickle
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import cv2
import numpy as np

# Part 1

## load dataset

In [70]:
def load_dataset():
  image_pickle_file_path = 'images.pkl'
  label_pickle_file_path = 'label.pkl'

  with open(image_pickle_file_path, 'rb') as file:
    images = pickle.load(file)

  with open(label_pickle_file_path, 'rb') as file:
    labels = pickle.load(file)

  return images, labels

In [71]:
n_images = 5

In [82]:
images, labels = load_dataset()
labels = np.array(labels)
random_indices = np.random.choice(560, size=n_images, replace=False)
# Extract the randomly selected values
random_values = images[random_indices]
random_labels = labels[random_indices]
from dataclasses import dataclass, field


@dataclass
class Images:
    image: np.ndarray = None
    original_features = []
    features = []
    clusters = None




## Proccess on images
- Extract features
- normalize features
- make features a single dimnetion vector

In [87]:
def extract_features(image):
    features = []
    original_features = []

    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # For each pixel in the image
    for i in range(img_hsv.shape[0]):
        for j in range(img_hsv.shape[1]):
            pixel = img_hsv[i, j]
            h, s, v = pixel
            x, y = i, j
            
            features.append((h, s, v, x, y))

    return features

In [88]:
def normalize_features(features):
    features_array = np.array(features)
    scaler = MinMaxScaler()
    normalized_features_array = scaler.fit_transform(features_array)
    normalized_features = normalized_features_array.tolist()
    return normalized_features

In [89]:
new_features = []

for img in random_values:
    features = extract_features(img)
    new_features.append(features)

In [90]:
# normalize features
for i, fimg in enumerate(new_features):
    new_features[i] = normalize_features(fimg)

In [91]:
def set_priority(arr, color, pos):
    color_columns = np.tile(arr[:, :3], (1, color))
    pos_columns = np.tile(arr[:, 3:], (1, pos))
    duplicated_arr = np.concatenate((color_columns, pos_columns), axis=1)

    return duplicated_arr

In [92]:
new_features_freq = []
for nf in new_features:
    new_features_freq.append(set_priority(np.array(nf), 4, 2).tolist())

## clustering

### k-means

In [93]:
k_number = 15
kmeans = KMeans(n_clusters=k_number)
new_clusters = []
for nf in new_features_freq:
    clusters = kmeans.fit_predict(nf)
    new_clusters.append(clusters)

c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\clus

In [94]:
for i, nc in enumerate(new_clusters):
    new_clusters[i] = nc.reshape(random_values.shape[1], random_values.shape[2])

## extracting clusters features for each image sepratedly

In [12]:
def find_matching_triads(array1, array2):   

    unique_values = np.unique(array1)
    result = []

    for value in unique_values:
        indexes = np.where(array1 == value)[0]
        triads = array2[indexes, :].tolist()
        result.append(triads)
        
    return result

### mean color for each cluster in each image

In [13]:
def calculate_mean(data):

    data_array = np.array(data)
    mean_values = np.mean(data_array, axis=0)
    mean_values = mean_values.astype(int)
    mean_values_list = mean_values.tolist()
    
    return mean_values_list

In [14]:
def mean_clusters_each_image(new_cls, random_values):

    all_images_mean_values = []
    
    for i, image_cls in enumerate(new_cls):

        one_image_clusters_mean_values = []
        reshaped_image_cls = image_cls.reshape(image_cls.shape[0] * image_cls.shape[1])
        reshaped_random_value = random_values[i].reshape(random_values.shape[1] * random_values.shape[2], random_values.shape[3])

        similar_clusters_pixels = find_matching_triads(reshaped_image_cls, reshaped_random_value)

        for smp in similar_clusters_pixels:
            one_image_clusters_mean_values.append(calculate_mean(smp))

        all_images_mean_values.append(one_image_clusters_mean_values)

    return all_images_mean_values

In [15]:
mean_each_cluster_each_image = mean_clusters_each_image(new_clusters, random_values)

### variance color for each cluster in each image

In [16]:
def calculate_var(data):

    data_array = np.array(data)
    var_values = np.var(data_array, axis=0)
    var_values = var_values.astype(int)
    var_values_list = var_values.tolist()
    
    return var_values_list

In [17]:
def var_clusters_each_image(new_cls, random_values):

    all_images_var_values = []


    for i, image_cls in enumerate(new_cls):

        one_image_clusters_var_values = []
        reshaped_image_cls = image_cls.reshape(image_cls.shape[0] * image_cls.shape[1])
        reshaped_random_value = random_values[i].reshape(random_values.shape[1] * random_values.shape[2], random_values.shape[3])

        similar_clusters_pixels = find_matching_triads(reshaped_image_cls, reshaped_random_value)

        for smp in similar_clusters_pixels:
            one_image_clusters_var_values.append(calculate_var(smp))

        all_images_var_values.append(one_image_clusters_var_values)

    return all_images_var_values

In [18]:
var_each_cluster_each_image = var_clusters_each_image(new_clusters, random_values)

### size of clusters in each image

In [19]:
def calculate_size(image_clusters):

    unique_clusters, counts = np.unique(image_clusters, return_counts=True)
    each_clusters_size = counts

    return each_clusters_size

In [20]:
size_each_cluster_each_image = []

for image_clusters in new_clusters:
    size_each_cluster_each_image.append(calculate_size(image_clusters))

## display clustering

In [95]:
def display_clustering(image_clusters, each_cluster_color, image_index):
    # Create an empty array to hold the HSV values for each pixel
    rgb_image = np.zeros((image_clusters.shape[0], image_clusters.shape[1], 3), dtype=np.uint8)

    # Assign the corresponding HSV values to each pixel
    for i in range(image_clusters.shape[0]):
        for j in range(image_clusters.shape[1]):
            value = image_clusters[i, j] 
            hsv = each_cluster_color[value]
            # rgb = colorsys.hsv_to_rgb(hsv[0] / 360, hsv[1] / 100, hsv[2] / 100)
            # rgb_image[i, j] = np.round(np.array(rgb) * 255).astype(int)
            rgb_image[i, j] = hsv

    # Display the image with corresponding RGB values using cv2.imshow
    cv2.imshow('org', random_values[image_index])
    cv2.imshow('Image with Corresponding RGB Values', cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR))

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [96]:
each_cluster_color = [
    [255, 0, 0],    # Red
    [0, 255, 0],    # Green
    [0, 0, 255],    # Blue
    [255, 255, 0],  # Yellow
    [255, 0, 255],  # Magenta
    [0, 255, 255],  # Cyan
    [128, 0, 0],    # Maroon
    [0, 128, 0],    # Green (Dark)
    [0, 0, 128],    # Navy
    [128, 128, 128], # Gray
    [255, 128, 0],  # Orange
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [128, 128, 0],  # Olive
    [192, 192, 192]  # Silver
]

In [97]:
for i, img_clusters in enumerate(new_clusters[:2]):
    display_clustering(img_clusters, each_cluster_color, i)

# Part 2

## create clusters feature vectors 

In [98]:
def extract_cluster_features(image_number, image_clusters, arr_new_features, k_number):
    clusters_features = []
    clusters = [[] for i in range(k_number)]
    for index, value in enumerate(image_clusters):
        clusters[value].append(arr_new_features[image_number, index])
    for cluster in clusters: # clusters contains all pixels which belong to the same cluster.
        temp_array = np.array(cluster)
        min_value = np.min(temp_array, axis=0)
        max_value = np.max(temp_array, axis=0)
        mean_value = np.mean(temp_array, axis=0)
        x_field = max_value[3] - min_value[3]
        y_field = max_value[4] - min_value[4]
        shape_field = np.array([x_field / y_field])
        combined_results = np.concatenate((min_value[:3], max_value[:3], mean_value[:3], shape_field), axis=None)
        clusters_features.append(combined_results)

    return np.array(clusters_features)

In [99]:
all_images_clusters_features = []
arr_new_features = np.array(new_features)
reshaped_image_clusters = image_clusters.reshape(image_clusters.shape[0] * image_clusters.shape[1])
for image_num, image_clusters in enumerate(new_clusters):
    all_images_clusters_features.append(extract_cluster_features(image_num, 
                                                                 reshaped_image_clusters,
                                                                 arr_new_features, k_number))

In [100]:
all_images_clusters_features

[array([[0.        , 0.        , 0.00515464, 0.99438202, 1.        ,
         1.        , 0.32416078, 0.5069726 , 0.48161164, 0.91346154],
        [0.06179775, 0.23921569, 0.01030928, 0.21348315, 1.        ,
         1.        , 0.12865256, 0.86001165, 0.42230884, 1.3539823 ],
        [0.        , 0.        , 0.07216495, 0.99438202, 1.        ,
         1.        , 0.18854079, 0.75639132, 0.77830017, 1.11702128],
        [0.        , 0.        , 0.03092784, 1.        , 1.        ,
         1.        , 0.4413635 , 0.28253174, 0.35608319, 0.61313869],
        [0.        , 0.        , 0.05670103, 0.98314607, 1.        ,
         1.        , 0.19292874, 0.7172818 , 0.49802726, 1.08115183],
        [0.        , 0.        , 0.03608247, 0.99438202, 1.        ,
         1.        , 0.21058445, 0.57815754, 0.57009857, 1.0334728 ],
        [0.        , 0.        , 0.05154639, 0.6741573 , 1.        ,
         1.        , 0.2207431 , 0.58397257, 0.59412635, 0.77917981],
        [0.08988764, 0.5333

## set priority for clusters features

In [101]:
color_pri = 5
pos_pri = 6

In [102]:
def set_priority_clusters_vectors(arr, color, pos):
    color_columns = np.tile(arr[:, :9], (1, color))
    pos_columns = np.tile(arr[:, 9:], (1, pos))
    duplicated_arr = np.concatenate((color_columns, pos_columns), axis=1)

    return duplicated_arr

In [103]:
all_images_clusters_features_freq = []
for nf in all_images_clusters_features:
    all_images_clusters_features_freq.\
            append(set_priority_clusters_vectors(np.array(nf), color_pri, pos_pri).tolist())

In [104]:
all_images_clusters_features_freq

[[[0.0,
   0.0,
   0.005154639175257714,
   0.9943820224719101,
   1.0,
   1.0,
   0.3241607785712791,
   0.5069725964334761,
   0.4816116418174412,
   0.0,
   0.0,
   0.005154639175257714,
   0.9943820224719101,
   1.0,
   1.0,
   0.3241607785712791,
   0.5069725964334761,
   0.4816116418174412,
   0.0,
   0.0,
   0.005154639175257714,
   0.9943820224719101,
   1.0,
   1.0,
   0.3241607785712791,
   0.5069725964334761,
   0.4816116418174412,
   0.0,
   0.0,
   0.005154639175257714,
   0.9943820224719101,
   1.0,
   1.0,
   0.3241607785712791,
   0.5069725964334761,
   0.4816116418174412,
   0.0,
   0.0,
   0.005154639175257714,
   0.9943820224719101,
   1.0,
   1.0,
   0.3241607785712791,
   0.5069725964334761,
   0.4816116418174412,
   0.9134615384615384,
   0.9134615384615384,
   0.9134615384615384,
   0.9134615384615384,
   0.9134615384615384,
   0.9134615384615384],
  [0.06179775280898876,
   0.2392156862745098,
   0.010309278350515483,
   0.21348314606741572,
   1.0,
   1.0,
   0

## clustering clusters

### k-means

In [105]:
k_number_2 = 100
kmeans_2 = KMeans(n_clusters=k_number_2)
arr_all_images_clusters_features_freq = np.array(all_images_clusters_features_freq)
arr_all_images_clusters_features_freq = arr_all_images_clusters_features_freq.reshape(\
                                            arr_all_images_clusters_features_freq.shape[0] *\
                                            arr_all_images_clusters_features_freq.shape[1],
                                            arr_all_images_clusters_features_freq.shape[2])

clusters2 = kmeans_2.fit_predict(arr_all_images_clusters_features_freq)
# kmeans = faissKMeans(d=all_images_clusters_features_freq.shape[1], k=k_number_2)
# kmeans.train(all_images_clusters_features_freq)

c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [106]:
clusters2_arr = clusters2.reshape(np.array(all_images_clusters_features_freq).shape[0], np.array(all_images_clusters_features_freq).shape[1])
clusters2_arr

array([[9, 3, 1, 0, 1, 1, 7, 3, 9, 0, 7, 3, 7, 7, 2],
       [9, 6, 9, 0, 9, 9, 0, 2, 9, 0, 0, 6, 0, 0, 6],
       [9, 2, 5, 0, 9, 5, 0, 3, 5, 4, 8, 6, 8, 0, 5],
       [5, 2, 5, 4, 5, 5, 4, 6, 5, 4, 4, 5, 4, 4, 5],
       [1, 2, 1, 7, 1, 1, 7, 3, 1, 7, 8, 2, 7, 7, 2]])

In [107]:
# create a ten value vector for each image
final_data = np.zeros((n_images, k_number_2))
final_data

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [108]:
def count_numbers(lst):
    # Initialize a list to store counts
    count_list = [0] * 10  # Index 0 is not used to align with numbers 1 to 10

    # Count occurrences of each number in the list
    for num in lst:
        if 1 <= num <= 10:
            count_list[num - 1] += 1  # Adjust index to align with numbers 1 to 10

    return count_list

In [109]:
histogram = np.empty((n_images, k_number_2))
for i, img_cls in enumerate(clusters2_arr):
    histogram[i] = count_numbers(img_cls)

histogram

array([[3., 1., 3., 0., 0., 0., 4., 0., 2., 0.],
       [0., 1., 0., 0., 0., 3., 0., 0., 5., 0.],
       [0., 1., 1., 1., 4., 1., 0., 2., 2., 0.],
       [0., 1., 0., 6., 7., 1., 0., 0., 0., 0.],
       [5., 3., 1., 0., 0., 0., 5., 1., 0., 0.]])

## classify

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Classify the datapoints with the Random Forest Classifier
def classify(datapoints, labels):
  test_size = 0.2
  X_train, X_test, y_train, y_test = train_test_split(datapoints, labels, test_size=test_size, random_state=42)

  clf = RandomForestClassifier(n_estimators=100, random_state=42)

  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  print(f"Accuracy: {accuracy * 100:.2f}%")

In [119]:
classify(histogram, random_labels)

Accuracy: 0.00%
